__качаем объявления о продаже квартир с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-11-07 10:37:33 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-07 10:37:33 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-07 10:37:37 | AdsListParser: downloader init
[INFO    ] 2022-11-07 10:37:37 | AvitoParser: init
[INFO    ] 2022-11-07 10:38:40 | AvitoParser: 72 pages for read
[INFO    ] 2022-11-07 10:38:40 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-07 10:38:43 | AdsListParser: read page 1
[INFO    ] 2022-11-07 10:38:47 | AdsListParser: read page 2
[INFO    ] 2022-11-07 10:38:50 | AdsListParser: read page 3
[INFO    ] 2022-11-07 10:38:52 | AdsListParser: read page 4
[INFO    ] 2022-11-07 10:38:57 | AdsListParser: read page 5
[INFO    ] 2022-11-07 10:39:00 | AdsListParser: read page 6
[INFO    ] 2022-11-07 10:39:03 | AdsListParser: read page 7
[INFO    ] 2022-11-07 10:39:09 | AdsListParser: read page 8
[INFO    ] 2022-11-07 10:40:15 | AdsListParser: read page 9
[INFO    ] 2022-11-07 10:40:19 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3572
3257


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
2897,2568500563,"Объявление «2-к. квартира, 51,9 м², 3/5 эт.» 11 фотографий",8000000,,"ул. Репина, 18",Объект №2054. \n\nХотите жить в самом сердце перспективного района с шаговой доступностью ко всей необходимой инфраструктуре для комфортной жизни? \n\nТогда трехкомнатная крупногабаритная квартира на Репина — для Вас! \n\nОсобенности Вашей новой квартиры: \n— Кос,64,2022-11-07 10:43:26.010265,sevastopol
712,2333405776,"Объявление «1-к. квартира, 31 м², 1/5 эт.» 11 фотографий",4500000,,"ул. Горпищенко, 43",Продам однокомнатную квартиру на первом этаже. Хорошее жилое состояние. Без балкона. Отличная инфраструктура. Прямая продажа. Торг.,16,2022-11-07 10:43:26.010265,sevastopol


In [11]:
# !pip install dash_table